In [1]:
import cv2
import torch
import numpy as np
from PIL import Image


/Users/kevinstoesser/opt/anaconda3/envs/torch-gpu/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
midas = torch.hub.load("intel-isl/MiDaS", "MiDaS")

use_large_model = True

if use_large_model:
    midas = torch.hub.load("intel-isl/MiDaS", "MiDaS")
else:
    midas = torch.hub.load("intel-isl/MiDaS", "MiDaS_small")

device = "cpu"
midas.to(device)

midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

if use_large_model:
    transform = midas_transforms.default_transform
else:
    transform = midas_transforms.small_transform


Using cache found in /Users/kevinstoesser/.cache/torch/hub/intel-isl_MiDaS_master


Loading weights:  None


Using cache found in /Users/kevinstoesser/.cache/torch/hub/facebookresearch_WSL-Images_main
Using cache found in /Users/kevinstoesser/.cache/torch/hub/intel-isl_MiDaS_master


Loading weights:  None


Using cache found in /Users/kevinstoesser/.cache/torch/hub/facebookresearch_WSL-Images_main
Using cache found in /Users/kevinstoesser/.cache/torch/hub/intel-isl_MiDaS_master


In [3]:
def depth(img):
    cv_image = np.array(img)
    img = cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB)

    input_batch = transform(img).to(device)
    with torch.no_grad():
        prediction = midas(input_batch)

        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode="bicubic",
            align_corners=False,
        ).squeeze()

    output = prediction.cpu().numpy()
    formatted = (output * 255 / np.max(output)).astype('uint8')
    img = Image.fromarray(formatted)
    return img


In [4]:
image_path = '/Users/kevinstoesser/Pictures/Jan_Rotholz/DSCF3616_neural_filter.jpg'
test_img = Image.open(image_path)
test_depth = depth(test_img)
test_depth.show()
test_depth = np.array(test_depth)


In [6]:
# Parameter
resize = True
resize_factor = 6

blur_factor = 10
target_depth = 1

# Read in the image and depth map
image = cv2.imread(image_path)
depth_map = test_depth

# resize for faster test inference
if resize:
    image = cv2.resize(
        image, [int(image.shape[1]/resize_factor), int(image.shape[0]/resize_factor)])
    depth_map = cv2.resize(
        depth_map,  [int(depth_map.shape[1]/resize_factor), int(depth_map.shape[0]/resize_factor)])
    print('image was resized')

# Convert the depth map to a floating point numpy array with values between 0 and 1
depth_map = depth_map / 255.0

# Create an empty output image of the same size as the input
output = np.zeros_like(image)

# Calculate the kernel sizes for all pixels based on the difference between their depth values and the target depth
kernel_sizes = (np.abs(depth_map - target_depth) * 10 + 1).astype(int)
# print(kernel_sizes.shape)
kernel_sizes[kernel_sizes % 2 == 0] += 1  # kernel size must be odd

# Create an empty output image of the same size as the input
output = np.zeros_like(image)

# Loop over each pixel in the image
for y in range(image.shape[0]):
    for x in range(image.shape[1]):
        # Get the kernel size for the current pixel
        kernel_size = kernel_sizes[y, x]

        # Apply a Gaussian blur to the image using the calculated kernel size
        blur = cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

        # Set the output pixel to the blurred value
        output[y, x] = blur[y, x]

# Save the output image
cv2.imwrite("output.jpg", output)


image was resized


True